In [1]:
!pip install tqdm
!pip install torch
!pip install torchvision torchaudio
!pip install fastai
!pip install tensorboardX
!pip install scikit-learn
!pip install "git+https://github.com/ncullen93/torchsample.git#egg=torchsample"
!pip install nibabel
!pip install nbdev

     |████████████████████████████████| 1.9 MB 4.4 MB/s eta 0:00:01
  Using cached torch-1.8.1-cp38-cp38-manylinux1_x86_64.whl (804.1 MB)
  Using cached torchvision-0.9.1-cp38-cp38-manylinux1_x86_64.whl (17.4 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.7.1
    Uninstalling torch-1.7.1:
      Successfully uninstalled torch-1.7.1
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.8.2
    Uninstalling torchvision-0.8.2:
      Successfully uninstalled torchvision-0.8.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.3.0 requires torch<1.8,>=1.7.0, but you have torch 1.8.1 which is incompatible.
fastai 2.3.0 requires torchvision<0.9,>=0.8, but you have torchvision 0.9.1 which is incompatible.
  Using cached torch-1.7.1-cp38-cp38-manylinux1_x86_64.whl (776.8 MB)
  Using cached torchvision-0.8.2-c

  Cloning https://github.com/ncullen93/torchsample.git to /tmp/pip-install-m0k4uomu/torchsample_636211e2539341c3bd7bf3fd8a6be50c
  Running command git clone -q https://github.com/ncullen93/torchsample.git /tmp/pip-install-m0k4uomu/torchsample_636211e2539341c3bd7bf3fd8a6be50c
  Created wheel for torchsample: filename=torchsample-0.1.3-py3-none-any.whl size=43415 sha256=55232d283dc286e463c5e8e37c4831c1d27d7357f71d8552b7656ca22579b338
  Stored in directory: /tmp/pip-ephem-wheel-cache-57w823td/wheels/54/14/cd/5612ec3bc0ce8e5b68b6a273873d18a3a5c817f35db70579a1
Successfully built torchsample
     |████████████████████████████████| 3.3 MB 4.7 MB/s eta 0:00:01
     |████████████████████████████████| 46 kB 2.2 MB/s eta 0:00:011
     |████████████████████████████████| 455 kB 6.7 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 2.3 MB/s  eta 0:00:01


  Attempting uninstall: nbconvert
    Found existing installation: nbconvert 6.0.7
    Uninstalling nbconvert-6.0.7:
      Successfully uninstalled nbconvert-6.0.7


In [13]:
!pip install torch==1.6.0 torchvision==0.7.0

     |████████████████████████████████| 748.8 MB 18 kB/s s eta 0:00:01     |█████████▏                      | 213.9 MB 13.1 MB/s eta 0:00:41     |██████████▉                     | 254.4 MB 11.7 MB/s eta 0:00:43     |█████████████████▊              | 414.2 MB 27.7 MB/s eta 0:00:13     |██████████████████████▎         | 520.5 MB 11.6 MB/s eta 0:00:20     |███████████████████████████     | 632.9 MB 807 kB/s eta 0:02:24
     |████████████████████████████████| 5.9 MB 16.6 MB/s eta 0:00:01
     |████████████████████████████████| 829 kB 18.9 MB/s eta 0:00:01
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491059 sha256=dfb222e553d90a18ab7df0371cc3aa10ede3e0caca522af5f2d886c8456baed9
  Stored in directory: /home/rob/.cache/pip/wheels/8e/70/28/3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4
Successfully built future
  Attempting uninstall: torch
    Found existing installation: torch 1.7.1
    Uninstalling torch-1.7.1:
      Successfully uninstalled torch-1.7.1
  Att

In [1]:
import shutil
import os
import time
from datetime import datetime
import argparse
import numpy as np
from tqdm import tqdm
from nbdev import *

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchsample.transforms import RandomRotate, RandomTranslate, RandomFlip, ToTensor, Compose, RandomAffine
from torchvision import transforms
import torch.nn.functional as F
from tensorboardX import SummaryWriter

from dataloader import MRDataset
import model

from sklearn import metrics
from fastai.vision.all import *

In [2]:
# parser.add_argument('-t', '--task', type=str, required=True,
#                     choices=['abnormal', 'acl', 'meniscus'])
# parser.add_argument('-p', '--plane', type=str, required=True,
#                     choices=['sagittal', 'coronal', 'axial'])
# parser.add_argument('--prefix_name', type=str, required=True)
# parser.add_argument('--augment', type=int, choices=[0, 1], default=1)
# parser.add_argument('--lr_scheduler', type=str,
#                     default='plateau', choices=['plateau', 'step'])
# parser.add_argument('--gamma', type=float, default=0.5)
# parser.add_argument('--epochs', type=int, default=50)
# parser.add_argument('--lr', type=float, default=1e-5)
# parser.add_argument('--flush_history', type=int, choices=[0, 1], default=0)
# parser.add_argument('--save_model', type=int, choices=[0, 1], default=1)
# parser.add_argument('--patience', type=int, default=5)
# parser.add_argument('--log_every', type=int, default=100)
# args = parser.parse_args()

class Args:
    def __init__(self):
        self.task = "abnormal"
        self.plane = "sagittal"
        self.prefix_name = "Test"
        self.augment = 1
        self.lr_scheduler = "plateau"
        self.gamma = 0.5
        self.epochs = 1
        self.lr = 1e-5
        self.flush_history = 0
        self.save_model =1
        self.patience = 5
        self.log_every = 100
        
args = Args()

In [3]:
args.task

'abnormal'

In [4]:
log_root_folder = "./logs/{0}/{1}/".format(args.task, args.plane)
if args.flush_history == 1:
    objects = os.listdir(log_root_folder)
    for f in objects:
        if os.path.isdir(log_root_folder + f):
            shutil.rmtree(log_root_folder + f)

now = datetime.now()
logdir = log_root_folder + now.strftime("%Y%m%d-%H%M%S") + "/"
os.makedirs(logdir)

writer = SummaryWriter(logdir)

augmentor = Compose([
    transforms.Lambda(lambda x: torch.Tensor(x)),
    RandomRotate(25),
    RandomTranslate([0.11, 0.11]),
    RandomFlip(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1, 1).permute(1, 0, 2, 3)),
])

train_dataset = MRDataset('./data/', args.task, args.plane, transform=augmentor, train=True)
validation_dataset = MRDataset('./data/', args.task, args.plane, train=False)

# train_loader = torch.utils.data.DataLoader(
#     train_dataset, batch_size=1, shuffle=True, num_workers=11, drop_last=False)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=18, drop_last=False)

# validation_loader = torch.utils.data.DataLoader(
#         validation_dataset, batch_size=1, shuffle=-True, num_workers=11, drop_last=False)

validation_loader = DataLoader(validation_dataset, batch_size=1, shuffle=False, num_workers=18, drop_last=False)


mrnet = model.MRNet()
bceloss = nn.BCEWithLogitsLoss()

./data/train-abnormal.csv


In [5]:
dls = DataLoaders(train_loader, validation_loader)

In [6]:
batch = dls.train.one_batch()
print(len(batch))
print(*[batch[i].shape for i in range(2)])

2
torch.Size([1, 20, 3, 256, 256]) torch.Size([1, 1, 2])


In [7]:
batch[1]

tensor([[[0., 1.]]])

In [8]:
doc(Learner)

In [6]:
learn = Learner(dls, mrnet, loss_func=bceloss, metrics=[RocAuc])

In [ ]:
learn.fine_tune(1)

epoch,train_loss,valid_loss,RocAuc,time
